In [ ]:
import jupyter_black

jupyter_black.load()
import pandas as pd
import matplotlib.pyplot as plt
import nfl_data_py as nfl
from src.visualisation.visualisation import plot_bar, plot_scatter
from src.utils.methods import flatten_grouped_cols
from src.visualisation.colors import team_unique_colors, team_unique_alt_colors
from src.utils.logos import get_team_logo
from matplotlib.offsetbox import AnnotationBbox
import matplotlib.ticker as mtick
import matplotlib.image as mpimg

pd.set_option("display.max_columns", None)

## Load Data

In [ ]:
pbp: pd.DataFrame = nfl.import_pbp_data([2023])

In [ ]:
# offense_players - list of id's
id_df = nfl.import_ids()
id_df.sample(4)

In [ ]:
wp_alpha = 0.05
pbp_pass = (
    pbp.query("week<=18")
    .query('play_type=="pass"')
    .query(f"wp >= {wp_alpha} and def_wp >={wp_alpha}")
)

# Target Shares

In [ ]:
[col for col in pbp_pass.columns if "yard" in col]

In [ ]:
target_df = (
    pbp_pass.groupby(["receiver_player_id", "receiver_player_name", "posteam"])
    .agg(
        {
            "play_id": "count",
            "complete_pass": "sum",
            "air_yards": "sum",
            "receiving_yards": "sum",
        }
    )
    .astype(int)
    .reset_index()
    .rename(
        columns={
            "play_id": "targets",
            "posteam": "team",
            "complete_pass": "receptions",
            "receiver_player_name": "player",
            "receiver_player_id": "player_id",
        }
    )
    .merge(
        id_df.dropna(subset=["gsis_id"])[["gsis_id", "position"]],
        how="left",
        left_on="player_id",
        right_on="gsis_id",
    )
    .drop(columns=["gsis_id"])
    .query("targets>=10")
    .sort_values("receptions", ascending=False)
)
target_df["team_targets"] = target_df.groupby("team")["targets"].transform("sum")
target_df["target_share"] = target_df["targets"] / target_df["team_targets"]
# Sort by team, then by target_share (descending), and then by receiving_yards (descending) to break any ties
target_df = target_df.sort_values(
    by=["team", "target_share", "receiving_yards"], ascending=[True, False, False]
)

# Rank each player within their team
target_df["target_share_rank"] = target_df.groupby("team").cumcount() + 1

target_df = target_df.sort_values("target_share", ascending=False)
target_df.head(24)

In [ ]:
target_df.query("target_share_rank<=3 and position!='WR'")

In [ ]:
target_df.query("position=='RB'").head(12)

In [ ]:
target_df.query("target_share_rank<=3").sample(8)

# Plot Target Share

In [ ]:
# Prepare data for plotting
# target_df = target_df.query("target_share_rank<=3")
teams = target_df["team"].unique().tolist()
bars_data = {}

for team in teams:
    team_data = target_df[
        (target_df["team"] == team) & (target_df["target_share_rank"] <= 3)
    ]
    bars_data[team] = team_data[["player", "position", "target_share"]].values
plt.style.use("default")
# Plotting
fig, ax = plt.subplots(figsize=(14, 12))

# Calculate total widths for sorting
team_total_widths = {}

for team, team_data in bars_data.items():
    cumulative_width = sum(team_data[:, 2])  # Sum of target_share for the top 3
    team_total_widths[team] = cumulative_width

# Sort teams by the total target share of the top 3 players
sorted_teams = sorted(teams, key=team_total_widths.get)  # type: ignore

for idx, team in enumerate(sorted_teams):
    cumulative_width = 0
    for player, position, target_share in bars_data[team]:
        # Draw the bar with a border (edgecolor) and the main fill color
        ax.barh(
            idx,
            target_share,
            left=cumulative_width,
            color=(team_unique_colors[team]),
            edgecolor=(team_unique_alt_colors[team]),
            linewidth=1.5,
            height=0.85,  # space between bars
        )
        # Add text
        ax.text(
            cumulative_width + target_share / 2,
            idx,
            f"{player} ({target_share*100:.1f}%)",
            va="center",
            ha="center",
            color=team_unique_alt_colors[team],
            fontsize=8,
            fontweight="heavy",  # Use "bold" or any other weight like "medium", "heavy"
        )
        cumulative_width += target_share
    team_total_widths[team] = cumulative_width
    # Add the team logo to the right of the bars
    ab = AnnotationBbox(
        get_team_logo(team, alpha=0.9, size=(25, 25)),
        (cumulative_width + 0.02, idx),  # Adjust the positioning as needed
        frameon=False,
    )
    ax.add_artist(ab)

# Customizing the plot
ax.set_yticks([])
ax.set_yticklabels([])  # Remove y-axis labels

# Format x-axis as percentages
ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# Add a note in the bottom right corner
fig.text(
    0.88,
    0.06,  # X and Y position in figure coordinates
    "Data: @nflfastR | Chart: @griffreichert",  # Text to display
    ha="right",  # Horizontal alignment
    va="top",  # Vertical alignment
    fontsize=8,  # Font size
    color="black",  # Text color
    # fontstyle="italic",  # Font style
)
ax.set_xlabel("Target Share (%)")
ax.set_title("Combined target share of the top three players on each team - 2023")
ax.set_ylim(-0.75, len(sorted_teams) - 0.25)
plt.show()

In [ ]:
plot_bar(
    target_df.head(20),
    x="target_share",
    y="player",
    citation=False,
    title="Target Share 2023",
)